In [8]:
import mediapipe as mp
import cv2
import numpy as np

# helps draw different detections from holistic models
mp_drawing = mp.solutions.drawing_utils

# import holistic models (this is just for point tracking with colors)
mp_holistic = mp.solutions.holistic

# mp pose
mp_pose = mp.solutions.pose


In [21]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS)
        
        # Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

        # Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

        # Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [6]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [17]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle

In [58]:
# BICEP CURLS

cap = cv2.VideoCapture(0)

# Curl counter variables
counterL = 0 
stageL = None

counterR = 0 
stageR = None

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks (LEFT ARM!!)
        try:
            landmarksL = results.pose_landmarks.landmark
            
            # Get coordinates
            shoulderL = [landmarksL[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarksL[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbowL = [landmarksL[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarksL[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wristL = [landmarksL[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarksL[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            
            # Calculate angle
            angleL = calculate_angle(shoulderL, elbowL, wristL)
            
            # Visualize angle
            cv2.putText(image, str(angleL), 
                           tuple(np.multiply(elbowL, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            
            # Curl counter logic
            if angleL > 160:
                stageL = "down"
            if angleL < 30 and stageL =='down':
                stageL="up"
                counterL +=1
                print(counterL)
                       
        except:
            pass
        
        # Extract landmarks (RIGHT ARM!)
        try:
            landmarksR = results.pose_landmarks.landmark
            
            # Get coordinates
            shoulderR = [landmarksR[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarksR[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            elbowR = [landmarksR[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarksR[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            wristR = [landmarksR[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarksR[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            
            # Calculate angle
            angleR = calculate_angle(shoulderR, elbowR, wristR)
            
            # Visualize angle
            cv2.putText(image, str(angleR), 
                           tuple(np.multiply(elbowR, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            
            # Curl counter logic
            if angleR > 160:
                stageR = "down"
            if angleR < 30 and stageR =='down':
                stageR="up"
                counterR +=1
                print(counterR)
                       
        except:
            pass
        
        # Render curl counter LEFT
        # Setup status box LEFT
        cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)
        
        # Render curl counter RIGHT
        # Setup status box RIGHT
        cv2.rectangle(image, (240,0), (465,73), (245,117,16), -1)
        
        
        # Rep data LEFT
        cv2.putText(image, 'LEFT REPS', (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counterL), 
                    (10,70), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        # Stage data LEFT
        cv2.putText(image, 'STAGE', (135,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 1, cv2.LINE_AA)
        cv2.putText(image, stageL, 
                    (90,70), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,0), 2, cv2.LINE_AA)
        
        # Rep data RIGHT
        cv2.putText(image, 'RIGHT REPS', (255,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counterR), 
                    (250,70), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        # Stage data RIGHT
        cv2.putText(image, 'STAGE', (375,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 1, cv2.LINE_AA)
        cv2.putText(image, stageR, 
                    (330,70), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,0), 2, cv2.LINE_AA)
        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

1
1
2
2
3
3
4
4
5
5
6
6
7
7
8
8
9
9
10
10


In [59]:
# JUMPING JACKS #

cap = cv2.VideoCapture(0)

# Curl counter variables
counter = 0 
stage = None

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks (LEFT ARM!!)
        try:
            landmarksL = results.pose_landmarks.landmark
            
            # Get coordinates
            shoulderL = [landmarksL[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarksL[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            elbowL = [landmarksL[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarksL[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            wristL = [landmarksL[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarksL[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            
            # Calculate angle
            angleL = calculate_angle(shoulderL, elbowL, wristL)
                       
        except:
            pass
        
        # Extract landmarks (RIGHT ARM!)
        try:
            landmarksR = results.pose_landmarks.landmark
            
            # Get coordinates
            shoulderR = [landmarksR[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarksR[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            elbowR = [landmarksR[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarksR[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            wristR = [landmarksR[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarksR[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            
            # Calculate angle
            angleR = calculate_angle(shoulderR, elbowR, wristR)
            
            # Visualize angle
            cv2.putText(image, str(angleR), 
                           tuple(np.multiply(elbowR, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            
            # Curl counter logic
            if angleR and angleL > 130:
                stage = "up"
            if angleR and angleL < 20 and stage =='up':
                stage="down"
                counter +=1
                print(counter)
                       
        except:
            pass
        
        # Render curl counter LEFT
        # Setup status box LEFT
        cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)

        
        # Rep data 
        cv2.putText(image, 'JUMP JACKS', (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), 
                    (10,70), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        # Stage data 
        cv2.putText(image, ' STAGE', (145,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 1, cv2.LINE_AA)
        cv2.putText(image, stage, 
                    (85,70), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,0), 2, cv2.LINE_AA)
        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
